In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Установим параметры
num_records = 1000000
duplicate_ratio = 0.1  # 10% дублей

# Генерация дат
date_range = pd.date_range(start='2018-01-01', end='2023-12-31', freq='H')
dates = np.random.choice(date_range, num_records)

# Генерация числовых данных
numeric_data = np.random.randn(num_records) * 100

# Генерация строковых данных
strings = [''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ', k=5)) for _ in range(num_records)]

# Создание датафрейма
df = pd.DataFrame({
    'datetime': dates,
    'numeric': numeric_data,
    'string': strings
})

# Добавление дубликатов
num_duplicates = int(num_records * duplicate_ratio)
duplicates = df.sample(num_duplicates, replace=True)
df = pd.concat([df, duplicates]).reset_index(drop=True)

# Сохранение в CSV
df.to_csv('synthetic_dataset_1000000.csv', index=False)


In [2]:
#pip install dask dask[dataframe]

In [3]:
#Считывание и процессинг

import dask.dataframe as dd
import pandas as pd

# Считывание данных с использованием dask
df = dd.read_csv('synthetic_dataset_1000000.csv', parse_dates=['datetime'])

# Удаление пустых / na строк
df = df.dropna()

# Удаление дубликатов
df = df.drop_duplicates()

# Строки, в которых нет цифр, превратить в пустые
df['string'] = df['string'].apply(lambda x: '' if not any(char.isdigit() for char in x) else x, meta=('string', 'object'))

# Удаление записей в промежутке от 1 до 3 часов ночи
df = df[~df['datetime'].dt.hour.isin([1, 2, 3])]

# Выполнение вычислений
df = df.compute()


In [4]:
# Преобразование pandas датафрейма в dask датафрейм с 8 разделами
ddf = dd.from_pandas(df, npartitions=8)

# Вычисление mean и median отдельно
numeric_mean = ddf.groupby(ddf['datetime'].dt.hour)['numeric'].mean().compute()

# Функция для вычисления median для каждой группы
def compute_median(group):
    return group.median()

# Применение функции к каждой группе
numeric_median = ddf.groupby(ddf['datetime'].dt.hour)['numeric'].apply(compute_median, meta=('numeric', 'float')).compute()

# Создание итогового датафрейма с результатами
numeric_aggregations = pd.DataFrame({
    'hour': numeric_mean.index,
    'numeric_mean': numeric_mean.values,
    'numeric_median': numeric_median.values
})

In [8]:
df_merged

,datetime,numeric,string,hour,unique_strings,mean_numeric,median_numeric,month
0,2019-03-11 09:00:00,-131.345393,,9,47,4.823313,14.275326,3
1,2023-12-27 09:00:00,132.668966,,9,47,4.823313,14.275326,12
2,2020-08-06 09:00:00,74.459865,,9,47,4.823313,14.275326,8
3,2022-10-11 09:00:00,34.888469,,9,47,4.823313,14.275326,10
4,2018-04-12 09:00:00,78.539039,,9,47,4.823313,14.275326,4
...,...,...,...,...,...,...,...,...
865,2021-01-22 16:00:00,-65.424040,,16,47,-28.834128,-20.632379,1
866,2022-09-22 16:00:00,-40.308902,,16,47,-28.834128,-20.632379,9
867,2021-04-04 16:00:00,47.874667,,16,47,-28.834128,-20.632379,4
868,2022-08-08 16:00:00,72.897169,,16,47,-28.834128,-20.632379,8


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import string
import seaborn as sns
import matplotlib.pyplot as plt

# Считаем уникальные значения для столбца 'string'
unique_strings = ddf[['datetime', 'string']].drop_duplicates().reset_index(drop=True)
unique_strings = unique_strings.groupby(unique_strings['datetime'].dt.hour).size().compute().reset_index(name='unique_strings')

# Выполняем группировку и агрегацию для столбца 'numeric' (среднее значение)
mean_numeric = ddf.groupby(ddf['datetime'].dt.hour)['numeric'].mean().compute().reset_index()
mean_numeric.columns = ['hour', 'mean_numeric']

# Вычисляем медиану отдельно
median_numeric = ddf.groupby(ddf['datetime'].dt.hour)['numeric'].apply(lambda x: x.quantile(0.5), meta=('numeric', 'f8')).compute().reset_index()
median_numeric.columns = ['hour', 'median_numeric']

# Объединяем результаты средней и медианной агрегации
numeric_agg = mean_numeric.merge(median_numeric, on='hour')

# Объединяем результаты с уникальными строками
agg_metrics = unique_strings.merge(numeric_agg, left_on='datetime', right_on='hour', how='inner')

# Объединение данных может создать дополнительный столбец, поэтому нужно проверить количество столбцов
# Переименование столбцов для соответствия SQL запросу
agg_metrics.columns = ['hour', 'unique_strings', 'hour_y', 'mean_numeric', 'median_numeric']
agg_metrics = agg_metrics.drop(columns=['hour_y'])

# Мерж с метриками
df_merged = pd.merge(df, agg_metrics, left_on=df['datetime'].dt.hour, right_on='hour')

# Аналитические метрики
# Гистограмма для колонки numeric
df_merged['numeric'].plot.hist(bins=20, title='Histogram of Numeric Column')
plt.show()

# 95% доверительный интервал для numeric
# Использование квантилей: Данный метод подходит для данных, которые не имеют строгого предположения о 
# нормальности распределения. Вычисление проводится путем нахождения квантилей данных: 2.5% и 97.5%.
# Это дает интервал, который содержит 95% всех значений выборки.
confidence_interval = np.percentile(df_merged['numeric'].dropna(), [2.5, 97.5])
print(f"95% доверительный интервал для numeric: [{confidence_interval[0]}, {confidence_interval[1]}]")

# Визуализация среднего значения numeric по месяцам
df_merged['month'] = df_merged['datetime'].dt.month
mean_numeric_monthly = df_merged.groupby('month')['numeric'].mean()
mean_numeric_monthly.plot(kind='bar', title='Mean Numeric Column by Month')
plt.xlabel('Month')
plt.ylabel('Mean Numeric Value')
plt.show()

# Heatmap по частотности символов в колонке string
heatmap_data = df_merged['string'].value_counts().reset_index()
heatmap_data.columns = ['string', 'frequency']
heatmap_data['length'] = heatmap_data['string'].apply(len)
heatmap_data.plot.hexbin(x='length', y='frequency', gridsize=15, cmap='YlGnBu', title='Heatmap of String Column')
plt.xlabel('String Length')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# SQL запрос для расчета метрик
# SELECT
#     EXTRACT(HOUR FROM datetime) AS hour,
#     COUNT(DISTINCT string) AS unique_strings,
#     AVG(numeric) AS mean_numeric,
#     MEDIAN(numeric) AS median_numeric
# FROM
#     synthetic_dataset
# WHERE
#     EXTRACT(HOUR FROM datetime) NOT IN (1, 2, 3)
# GROUP BY
#     EXTRACT(HOUR FROM datetime)
# ORDER BY
#     hour;